In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import plotly.plotly as py
import plotly.graph_objs as go
import plotly

In [2]:
import os
os.listdir()

['.ipynb_checkpoints',
 'fighter_interview.csv',
 'event_interview.csv',
 'fighter_analysis.ipynb',
 'outcome_interview.csv']

In [3]:
# Reading data files

outcome_file = pd.read_csv('outcome_interview.csv')
event_file = pd.read_csv('event_interview.csv')
fighter_file = pd.read_csv('fighter_interview.csv')

In [4]:
outcome_file.head()

,id,fighter1_id,fighter2_id,winner_id,event_id
0,2264,803,138,138.0,18
1,2272,3025,466,466.0,494
2,2280,3005,1671,1671.0,375
3,2292,2110,284,284.0,352
4,2310,1346,607,1346.0,826


In [5]:
event_file.head()

,id,name,date,promoter,location,attendance,created_at,modified_at
0,1,One FC 10: Champions and Warriors,2013-09-13,OneFC,NaN,NaN,2019-04-01T19:56:24.477518,2019-04-01T19:56:24.477518
1,2,Legacy FC 45: Todhunter vs. Smith,2015-09-11,Legacy Fc,NaN,NaN,2019-04-01T19:56:28.403410,2019-04-01T19:56:28.403410
2,3,LFA 27: Watley vs. Wilson,2017-11-10,LFA,NaN,NaN,2019-04-01T19:56:32.488043,2019-04-01T19:56:32.488043
3,4,ONE FC 21: Roar of Tigers,2014-10-17,OneFC,NaN,NaN,2019-04-01T19:56:39.395144,2019-04-01T19:56:39.395144
4,5,One FC 8: Kings and Champions,2013-04-05,OneFC,NaN,NaN,2019-04-01T19:56:39.404131,2019-04-01T19:56:39.404131


In [6]:
fighter_file.head()

,id,name,nationality
0,111,Bertrand Amoussou,France
1,3227,Dave Zitanick,United States
2,2002,Jim Mullen,United States
3,2033,Mario Neto,Brazil
4,1667,Aaron Lanfranco,United States


In [7]:
# Map Fighter1 name and country
map_file = pd.merge(outcome_file,fighter_file,how = 'left',left_on = 'fighter1_id' , right_on = 'id')

# Map Fighter2 name and country
map_file = pd.merge(map_file,fighter_file,how = 'left' , left_on = 'fighter2_id' , right_on = 'id')

# Map event information
map_file = pd.merge(map_file , event_file[['id','name']] , how = 'left' , left_on = 'event_id' , right_on = 'id')

# Filter required columns
map_file = map_file[['name_x' , 'name_y', 'nationality_x' , 'nationality_y' , 'name']]

# Rename columns
map_file.rename(index = str , columns = {'name_x' : 'fighter1_name',
                                        'name_y' : 'fighter2_name',
                                        'nationality_x' : 'fighter1_nationality',
                                        'nationality_y' : 'fighter2_nationality',
                                        'name' : 'event_name'} , inplace = True)

In [8]:
map_file.head()

,fighter1_name,fighter2_name,fighter1_nationality,fighter2_nationality,event_name
0,Mike Easton,Raphael Assuncao,United States,Brazil,UFC on FOX 5: Henderson vs. Diaz
1,Bobby Voelker,Patrick Cote,United States,Canada,UFC 158: St-Pierre vs. Diaz
2,Jamie Varner,Joe Lauzon,United States,United States,UFC on FOX 4: Shogun vs. Vera
3,Andy Ogle,Maximo Blanco,England,Venezuela,UFC Fight Night 41: Munoz vs. Mousasi
4,Anthony Johnson,Phil Davis,United States,United States,UFC 172: Jones vs. Teixeira


In [9]:
# Create distribution of nationalities

# combining the two columns to create a counter object

out1 = Counter(map_file['fighter1_nationality'])
out2 = Counter(map_file['fighter2_nationality'])
fight_dist = out1 + out2


# Importing an external database of country and codes to help in plotting
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')

# Combining with our counts data
image_df = pd.DataFrame.from_dict(fight_dist,orient='index').reset_index()
image_df = image_df.merge(df[['COUNTRY','CODE']],how='left',left_on = 'index',right_on = 'COUNTRY')


In [15]:

# Plotting nationalities on the world map using plotly

# Setting up API
plotly.tools.set_credentials_file(username='abhishekshettynov19', api_key='ViGFMPIsV6p8Ett6rHC7')


data = [go.Choropleth(
locations = image_df['CODE'],
z = image_df[0],
text = image_df['index'],
colorscale = [
        [0.0, 'rgb(165,0,38)'], 
        [0.1111111111111111, 'rgb(215,48,39)'], 
        [0.2222222222222222, 'rgb(244,109,67)'],
        #[0.3333333333333333, 'rgb(253,174,97)'], 
        #[0.4444444444444444, 'rgb(254,224,144)'], 
        [0.5555555555555556, 'rgb(224,243,248)'],
        [0.7777777777777777, 'rgb(171,217,233)'],
        [0.8888888888888888, 'rgb(116,173,209)'], 
        [0.9555555555555555, 'rgb(69,117,180)'],
        [1.0, 'rgb(49,54,149)']
    ],
    autocolorscale = False,
    reversescale = True,
    marker = go.choropleth.Marker(
        line = go.choropleth.marker.Line(
            color = 'rgb(180,180,180)',
            width = 0.5
        )),
    colorbar = go.choropleth.ColorBar(
        tickprefix = '',
        title = 'Count<br>Fighters'),
)]

layout = go.Layout(
    title = go.layout.Title(
        text = 'Events'
    ),
    geo = go.layout.Geo(
        showframe = False,
        showcoastlines = False,
        projection = go.layout.geo.Projection(
            type = 'equirectangular'
        )
    ),
    annotations = [go.layout.Annotation(
        x = 0.55,
        y = 0.1,
        xref = 'paper',
        yref = 'paper',
        text = 'Source: <a href="https://www.cia.gov/library/publications/the-world-factbook/fields/2195.html">\
            CIA World Factbook</a>',
        showarrow = False
    )]
)

fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename = 'd3-world-map')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~abhishekshettynov19/0 or inside your plot.ly account where it is named 'd3-world-map'


## US has the maximum number of fighters followed by Brazil , Canada , Russia , Japan and then the rest of the world

In [16]:
map_file.to_csv('Output.csv' , index = False)